In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sp
from scipy.integrate import odeint
import os

In [32]:
os.getcwd()

'C:\\Users\\ben.lucke\\Desktop\\projects\\sandbox'

In [24]:
data = 'altimeter_data.csv'
df = pd.read_csv(data,header=None, names=['t','yk','hk','sk','cbk'])

In [25]:
h, s, d, delt, cb, rn, hr, hn, re, gn = sp.symbols(r'h s d \Delta{t} C_b \rho_0 h_\rho h_0 R_E g_0')

In [26]:
A_discrete_symbolic = sp.Matrix([[h + delt*s],
                                [s + (delt*(((rn*s**2)/(2*cb)*sp.exp(-h/hr)) - (gn*((re/(re+h))**2))))],
                                [cb]])
constants_mapper = dict(zip([delt,rn,gn,hr,re],[.5, .0765, 32.2, 30000, 20902260]))
A_jacobian_symbolic = A_discrete_symbolic.jacobian([h, s, cb]).subs(constants_mapper) # Sub in constants

In [27]:
P0 = np.diag([100**2, 10**2, 1])
Q = np.diag([10**2, 10**2, .05**2])
R = 100**2

### Dynamics Model

In [28]:
def propogate(x):
    '''Function to compute dynamics of state variables'''
    dt = 0.5
    rho_0 = .0765
    g0 = 32.2
    h_rho = 30000
    Re = 20902260
    h = x[0]
    s = x[1]
    cb = x[2]

    hkp1 = h + dt*s
    skp1 = s+ dt*(((rho_0*s**2)/(2*cb)*np.exp(-h/h_rho)) - (g0*((Re/(Re+h))**2)))
    cbkp1 = cb
    return hkp1, skp1, cbkp1

In [29]:
def calcF(A_jac, x):
    '''Function that computes analytical jacobian of symbolic jacobian of A'''
    x = x.flatten()
    h_in = x[0]
    s_in = x[1]
    cb_in = x[2]
    F_out = A_jac.subs({h:h_in, s:s_in, cb:cb_in})
    return np.array(F_out).astype(np.float64)

### Measurement Model

In [109]:
H_discrete_symbolic = sp.Matrix([sp.sqrt(d**2 + h**2)])
H_jacobian_symbolic = H_discrete_symbolic.jacobian([h,s,cb])

In [169]:
def get_measurement(x):
    h_in = x[0]
    s_in = x[1]
    cb_in = x[2]
    d = 100000
    out = np.sqrt(d**2 + h_in**2)
    return out

### Propogate Dynamics

In [176]:
x0 = np.array([400e3, -2000, 20])

In [177]:
n = 600
res = np.zeros((n, 3))
mmts = np.zeros((n,1))
for k in range(n):
    if k == 0:
        x = x0
        P = P0
        meas = get_measurement(x)
        res[k] = x
        mmts[k] = meas
        continue
    hk, sk, cbk = propogate(x)
    x = np.array([hk, sk, cbk])
    res[k] = x
    mmts[k] = get_measurement(x)